# Load Forecasting

## 부하 예측을 위한 데이터 수집 및 변환

### 부하 데이터 프로세싱

In [9]:
import pandas as pd

### 부하 데이터 불러오기

In [2]:
raw_gen_data = pd.read_excel('1.Data/raw_generation_data.xlsx')
raw_gen_data.head()

,Date,00,01,02,03,04,05,06,07,08,...,14,15,16,17,18,19,20,21,22,23
0,2018-01-01,62446,59691,57231,55500,54821,55000,55641,56212,55816,...,52206,52503,53553,55466,58560,60360,60588,60408,60333,61328
1,2018-01-02,62145,59313,56977,55828,55675,56392,58196,61468,66824,...,76154,77071,77019,77821,77781,75982,73583,71269,69243,68858
2,2018-01-03,70214,66714,64267,62935,62446,63043,64539,67464,72310,...,77396,78613,78719,80122,80242,78515,76217,73977,71968,71338
3,2018-01-04,72583,68811,66239,64792,64326,64771,66142,68994,73706,...,79590,80954,81231,82344,81784,79743,76962,74045,71729,71174
4,2018-01-05,72427,68787,66141,64752,64128,64570,65695,68262,72565,...,76755,77557,77869,79366,79199,77443,74884,72530,70450,69942


### stack을 활용하여 열을 행으로 변경

### **pandas.DataFrame.stack**

pandas의 stack 함수는 데이터의 구조를 재조정하는 데 사용되는 매우 유용한 도구입니다. 이 함수는 DataFrame의 컬럼들을 로우로 "쌓아올려" MultiIndex를 가진 Series로 변환합니다. 이 과정을 이해하기 위해 간단한 예제를 살펴보겠습니다.

**예제1**

다음과 같은 DataFrame을 가지고 있다고 가정해봅시다:

|   | A | B |
|---|---|---|
| 0 | 1 | 4 |
| 1 | 2 | 5 |
| 2 | 3 | 6 |

이 DataFrame에 **'stack'** 함수를 적용하면 결과는 아래와 같습니다.

|   |   |   |
|---|---|---|
| 0 | A | 1 |
| 0 | B | 4 |
| 1 | A | 2 |
| 1 | B | 5 |
| 2 | A | 3 |
| 2 | B | 6 |

여기서 주목해야 할 점은 결과가 MultiIndex를 가진 Series임을 확인할 수 있습니다. 첫 번째 레벨의 인덱스는 원본 DataFrame의 로우 인덱스이며, 두 번째 레벨의 인덱스는 원본 DataFrame의 컬럼입니다.

**예제2**

다음과 같은 다중 레벨 컬럼을 가진 DataFrame을 고려해봅시다:

|   |     |  A  |  A  |  B  |  B  |
|---|-----|-----|-----|-----|-----|
|   |     | X   | Y   | X   | Y   |
|---|-----|-----|-----|-----|-----|
| 0 |     | 1   | 2   | 3   | 4   |
| 1 |     | 5   | 6   | 7   | 8   |


이 경우, level 매개변수를 사용하여 어떤 레벨을 로우로 쌓을지 지정할 수 있습니다. 예를 들어, 최상위 레벨을 로우로 쌓아보겠습니다:

stacked_level_0 = df.stack(level=0)

결과는:

|   |   | X | Y |
|---|---|---|---|
| 0 | A | 1 | 2 |
| 0 | B | 3 | 4 |
| 1 | A | 5 | 6 |
| 1 | B | 7 | 8 |

stack 함수는 데이터를 재구조화하는 데 있어 강력한 도구로, 데이터 분석 및 변환 작업에서 자주 사용됩니다.

In [3]:
stacked_data = raw_gen_data.set_index('Date').stack().reset_index()
stacked_data

,Date,level_1,0
0,2018-01-01,00,62446
1,2018-01-01,01,59691
2,2018-01-01,02,57231
3,2018-01-01,03,55500
4,2018-01-01,04,54821
...,...,...,...
22627,2020-07-31,19,70347
22628,2020-07-31,20,70092
22629,2020-07-31,21,69176
22630,2020-07-31,22,66569


In [4]:
stacked_data.columns = ['Date', 'Hour', 'Generation']
stacked_data

,Date,Hour,Generation
0,2018-01-01,00,62446
1,2018-01-01,01,59691
2,2018-01-01,02,57231
3,2018-01-01,03,55500
4,2018-01-01,04,54821
...,...,...,...
22627,2020-07-31,19,70347
22628,2020-07-31,20,70092
22629,2020-07-31,21,69176
22630,2020-07-31,22,66569


### **pandas.DataFrame.datetime**

pandas는 날짜와 시간을 효과적으로 처리하기 위한 다양한 도구와 기능을 제공합니다. datetime은 Python의 기본 라이브러리 중 하나로, 날짜와 시간을 다루는 데 사용됩니다. pandas는 이 datetime 모듈을 기반으로 확장된 기능들을 제공하며, 특히 Timestamp 객체와 관련 메서드들을 통해 강력한 시계열 분석 기능을 지원합니다.

pandas의 pd.to_datetime 함수는 다양한 형식의 날짜와 시간 문자열을 pandas Timestamp 객체로 변환하는데 사용됩니다.

In [5]:
stacked_data['Time'] = pd.to_datetime(stacked_data['Date'].astype(str) + ' ' + stacked_data['Hour'].astype(str) + ':00:00')
stacked_data

,Date,Hour,Generation,Time
0,2018-01-01,00,62446,2018-01-01 00:00:00
1,2018-01-01,01,59691,2018-01-01 01:00:00
2,2018-01-01,02,57231,2018-01-01 02:00:00
3,2018-01-01,03,55500,2018-01-01 03:00:00
4,2018-01-01,04,54821,2018-01-01 04:00:00
...,...,...,...,...
22627,2020-07-31,19,70347,2020-07-31 19:00:00
22628,2020-07-31,20,70092,2020-07-31 20:00:00
22629,2020-07-31,21,69176,2020-07-31 21:00:00
22630,2020-07-31,22,66569,2020-07-31 22:00:00


In [6]:
final_data_using_stack = stacked_data[['Time', 'Generation']].sort_values(by='Time').reset_index(drop=True)
final_data_using_stack

,Time,Generation
0,2018-01-01 00:00:00,62446
1,2018-01-01 01:00:00,59691
2,2018-01-01 02:00:00,57231
3,2018-01-01 03:00:00,55500
4,2018-01-01 04:00:00,54821
...,...,...
22627,2020-07-31 19:00:00,70347
22628,2020-07-31 20:00:00,70092
22629,2020-07-31 21:00:00,69176
22630,2020-07-31 22:00:00,66569


### 날씨 데이터 프로세싱

**os 모듈**

os 모듈은 Python의 표준 라이브러리에 포함되어 있으며, 운영 체제와 상호 작용하기 위한 다양한 기능을 제공합니다. 이 모듈을 사용하면 디렉토리 구조를 탐색하거나, 파일을 생성/삭제하거나, 환경 변수에 접근하는 등의 작업을 수행할 수 있습니다.

주요 기능들:

* os.listdir(path): 주어진 경로의 디렉토리 내에 있는 모든 파일 및 하위 디렉토리의 이름을 리스트로 반환합니다.
* os.path.join(path1, path2, ...): 하나 이상의 경로 구성 요소를 연결하여 단일 경로 문자열을 생성합니다. 이 함수는 운영 체제의 디렉토리 구분 기호 (Windows에서는 '\'와 같이)를 사용하여 경로를 연결합니다.
* os.path.exists(path): 주어진 경로가 존재하는지 확인합니다.
* os.mkdir(path): 주어진 경로에 디렉토리를 생성합니다.
* os.remove(path): 주어진 경로의 파일을 삭제합니다.

이 외에도 os 모듈은 파일 및 디렉토리에 관한 다양한 정보와 작업을 위한 많은 함수와 속성을 제공합니다.

**os.listdir()**

**개요**

os.listdir() 함수는 os 모듈의 일부로, 주어진 디렉토리의 모든 파일과 하위 디렉토리의 이름을 리스트로 반환합니다.

**사용법**

```
import os

file_list = os.listdir(path)

```

여기서 path는 파일 및 하위 디렉토리의 이름을 가져오려는 디렉토리의 경로입니다.

**예제**

예를 들어, 현재 작업 디렉토리의 모든 파일과 폴더를 나열하려면 다음과 같이 사용할 수 있습니다:

```
import os

current_directory = os.getcwd()  # 현재 작업 디렉토리의 경로를 가져옵니다.
file_list = os.listdir(current_directory)

print(file_list)

```

**주의 사항**

os.listdir()는 숨겨진 파일과 폴더도 포함하여 반환합니다.
반환된 파일 및 폴더의 목록은 특정한 순서를 가지지 않을 수 있으므로, 특정한 순서대로 정렬이 필요한 경우 추가적인 코드를 작성해야 합니다.


**os.path.join()**

os.path.join()은 os.path 모듈의 함수로, 여러 개의 경로 구성 요소를 연결하여 하나의 경로 문자열을 형성합니다. 이 함수는 운영 체제에 따라 적절한 디렉토리 구분자 (예: Windows에서는 '\', UNIX에서는 '/')를 사용하여 경로 구성 요소를 연결합니다.

**사용 방법**

하나 이상의 경로 구성 요소를 연결하여 완전한 경로를 형성하려면, 각 경로 구성 요소를 os.path.join()의 인자로 전달합니다.

**예시**

기본 디렉토리 경로와 하위 디렉토리 및 파일 이름을 연결하여 전체 파일 경로를 형성하려면 다음과 같이 사용할 수 있습니다:

```
file_path = os.path.join("base_directory", "sub_directory", "filename.txt")
```

이 코드는 "base_directory/sub_directory/filename.txt" 또는 "base_directory\sub_directory\filename.txt"와 같은 경로 문자열을 반환합니다. (운영 체제에 따라 다름)

**주의 사항**
os.path.join()은 주어진 운영 체제의 디렉토리 구분자를 사용하여 경로 구성 요소를 연결합니다. 이렇게 하면 동일한 코드가 다양한 운영 체제에서 올바르게 작동하게 됩니다.
연결하려는 경로 구성 요소 중 하나가 절대 경로인 경우 (예: /path/to/directory 또는 C:\path\to\directory), os.path.join()은 해당 절대 경로부터 경로를 형성합니다.


In [ ]:
import os

In [14]:
files = [f for f in os.listdir('1.Data/raw_data') if 'Weather' in f and f.endswith('.xlsx')]
files

['Incheon_Weather.xlsx',
 'Busan_Weather.xlsx',
 'Gwangju_Weather.xlsx',
 'Daejeon_Weather.xlsx',
 'Daegu_Weather.xlsx',
 'Seoul_Weather.xlsx']

In [13]:
dfs = []

for file in files:
    city_name = file.split('_')[0]

    df = pd.read_excel(os.path.join('1.Data/raw_data', file), index_col=0)

    rename_dict = {col: city_name + "_" + col for col in df.columns if col != "Time"}
    df.rename(columns=rename_dict, inplace=True)

    dfs.append(df)

**pandas.merge()**

pandas.merge()는 pandas 라이브러리의 핵심 함수 중 하나로, 두 데이터프레임을 하나 이상의 키를 기준으로 병합합니다. 이 함수는 SQL의 JOIN 연산과 유사한 작업을 수행합니다.

**사용 방법**

두 데이터프레임을 병합하기 위해 pandas.merge() 함수를 사용할 수 있습니다. 병합하려는 열의 이름을 on 매개변수로 지정하며, 병합 방식은 how 매개변수를 통해 지정할 수 있습니다.

**예시**

두 데이터프레임, df1과 df2,가 있고 이들을 'key' 열을 기준으로 내부 병합하려면 다음과 같이 사용할 수 있습니다:

```
merged_df = pd.merge(df1, df2, on='key', how='inner')

```

**주요 매개변수**

* on: 병합할 열의 이름 또는 이름의 리스트. 두 데이터프레임 모두에 존재해야 합니다.

* how: 병합 방식을 지정하는 문자열. 다음 값 중 하나를 선택할 수 있습니다:
    * 'left': 왼쪽 데이터프레임의 키를 기준으로 병합합니다.
    * 'right': 오른쪽 데이터프레임의 키를 기준으로 병합합니다.
    * 'inner': 두 데이터프레임에 모두 존재하는 키를 기준으로 병합합니다.
    * 'outer': 두 데이터프레임 중 하나에만 존재하는 키도 포함하여 병합합니다.
    * left_on 및 right_on: 두 데이터프레임의 열 이름이 다를 경우 각각의 데이터프레임에서 사용할 열 이름을 지정합니다.


**예제 데이터:**
df1:
| key | A  |
|-----|----|
| K1  | A1 |
| K2  | A2 |
| K3  | A3 |

df2:
| key | B  |
|-----|----|
| K2  | B2 |
| K3  | B3 |
| K4  | B4 |

1. inner

두 데이터프레임에 모두 존재하는 키를 기준으로 병합합니다.

결과:

| key | A  | B  |
|-----|----|----|
| K2  | A2 | B2 |
| K3  | A3 | B3 |


2. left

왼쪽 데이터프레임의 키를 기준으로하여 병합하며, 오른쪽 데이터프레임의 해당 키가 없는 경우 NaN 값으로 채워집니다.

결과:

| key | A  | B   |
|-----|----|-----|
| K1  | A1 | NaN |
| K2  | A2 | B2  |
| K3  | A3 | B3  |

3. right

오른쪽 데이터프레임의 키를 기준으로 병합하며, 왼쪽 데이터프레임의 해당 키가 없는 경우 NaN 값으로 채워집니다.

결과:

| key | A   | B  |
|-----|-----|----|
| K2  | A2  | B2 |
| K3  | A3  | B3 |
| K4  | NaN | B4 |

4. outer

두 데이터프레임 중 하나에만 존재하는 키도 포함하여 병합합니다. 해당 키가 없는 데이터프레임의 값은 NaN으로 채워집니다.

결과:

| key | A   | B   |
|-----|-----|-----|
| K1  | A1  | NaN |
| K2  | A2  | B2  |
| K3  | A3  | B3  |
| K4  | NaN | B4  |

**주의 사항**

병합 시 중복된 열 이름이 있는 경우, suffixes 매개변수를 사용하여 중복을 해결할 수 있습니다.
병합할 열의 데이터 타입이 일치해야 합니다. 그렇지 않으면 오류가 발생할 수 있습니다.

In [15]:
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="Time", how="outer")

merged_df

,Time,Incheon_Temp,Incheon_Precipitation,Incheon_Humidity,Incheon_Insolation,Incheon_Cloud,Busan_Temp,Busan_Precipitation,Busan_Humidity,Busan_Insolation,...,Daegu_Temp,Daegu_Precipitation,Daegu_Humidity,Daegu_Insolation,Daegu_Cloud,Seoul_Temp,Seoul_Precipitation,Seoul_Humidity,Seoul_Insolation,Seoul_Cloud
0,2018-01-01 00:00:00,-1.4,0.1,49.0,0.01,0.0,1.2,0.0,35.0,NaN,...,0.8,0.0,37.0,0.00,0.0,-3.2,0.0,40.0,0.01,0.0
1,2018-01-01 01:00:00,-1.6,0.1,47.0,0.00,0.0,NaN,0.0,33.0,0.02,...,0.1,0.0,43.0,0.00,0.0,-3.3,0.0,41.0,0.00,0.0
2,2018-01-01 02:00:00,-1.9,0.1,47.0,0.00,0.0,0.1,0.0,34.0,0.02,...,0.0,0.0,46.0,0.01,0.0,-3.7,0.0,42.0,0.00,0.0
3,2018-01-01 03:00:00,-1.8,0.1,44.0,0.00,0.0,0.0,0.0,37.0,0.02,...,-0.1,0.0,47.0,NaN,0.0,-4.0,0.0,44.0,0.00,0.0
4,2018-01-01 04:00:00,-2.0,NaN,48.0,0.00,0.0,-0.1,0.0,42.0,0.02,...,NaN,0.0,47.0,0.01,0.0,-4.2,NaN,53.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,2020-07-31 19:00:00,25.0,NaN,95.0,0.18,10.0,26.1,0.0,86.0,0.22,...,27.8,0.0,73.0,0.25,10.0,27.4,0.3,82.0,NaN,9.0
22628,2020-07-31 20:00:00,24.9,0.0,96.0,0.02,10.0,NaN,0.0,84.0,0.01,...,27.6,0.0,NaN,0.02,10.0,27.0,0.3,83.0,0.09,9.0
22629,2020-07-31 21:00:00,24.9,0.0,96.0,0.02,10.0,NaN,0.0,84.0,0.01,...,27.0,NaN,71.0,0.02,8.0,26.7,0.2,86.0,0.09,NaN
22630,2020-07-31 22:00:00,24.9,0.0,97.0,0.02,10.0,26.1,0.0,85.0,0.01,...,26.4,0.0,75.0,0.02,10.0,26.5,0.2,87.0,0.09,NaN


In [27]:
merged_df[merged_df.columns[1:]] = merged_df[merged_df.columns[1:]].interpolate(method='linear')
merged_df

,Time,Incheon_Temp,Incheon_Precipitation,Incheon_Humidity,Incheon_Insolation,Incheon_Cloud,Busan_Temp,Busan_Precipitation,Busan_Humidity,Busan_Insolation,...,Daegu_Temp,Daegu_Precipitation,Daegu_Humidity,Daegu_Insolation,Daegu_Cloud,Seoul_Temp,Seoul_Precipitation,Seoul_Humidity,Seoul_Insolation,Seoul_Cloud
0,2018-01-01 00:00:00,-1.4,0.1,49.0,0.01,0.0,1.20,0.0,35.0,NaN,...,0.8,0.0,37.0,0.00,0.0,-3.2,0.0,40.0,0.010,0.000000
1,2018-01-01 01:00:00,-1.6,0.1,47.0,0.00,0.0,0.65,0.0,33.0,0.02,...,0.1,0.0,43.0,0.00,0.0,-3.3,0.0,41.0,0.000,0.000000
2,2018-01-01 02:00:00,-1.9,0.1,47.0,0.00,0.0,0.10,0.0,34.0,0.02,...,0.0,0.0,46.0,0.01,0.0,-3.7,0.0,42.0,0.000,0.000000
3,2018-01-01 03:00:00,-1.8,0.1,44.0,0.00,0.0,0.00,0.0,37.0,0.02,...,-0.1,0.0,47.0,0.01,0.0,-4.0,0.0,44.0,0.000,0.000000
4,2018-01-01 04:00:00,-2.0,0.1,48.0,0.00,0.0,-0.10,0.0,42.0,0.02,...,-0.4,0.0,47.0,0.01,0.0,-4.2,0.0,53.0,0.000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,2020-07-31 19:00:00,25.0,0.0,95.0,0.18,10.0,26.10,0.0,86.0,0.22,...,27.8,0.0,73.0,0.25,10.0,27.4,0.3,82.0,0.175,9.000000
22628,2020-07-31 20:00:00,24.9,0.0,96.0,0.02,10.0,26.10,0.0,84.0,0.01,...,27.6,0.0,72.0,0.02,10.0,27.0,0.3,83.0,0.090,9.000000
22629,2020-07-31 21:00:00,24.9,0.0,96.0,0.02,10.0,26.10,0.0,84.0,0.01,...,27.0,0.0,71.0,0.02,8.0,26.7,0.2,86.0,0.090,8.666667
22630,2020-07-31 22:00:00,24.9,0.0,97.0,0.02,10.0,26.10,0.0,85.0,0.01,...,26.4,0.0,75.0,0.02,10.0,26.5,0.2,87.0,0.090,8.333333


In [ ]:
for col in merged_df.columns:
    if 'Cloud' in col:
        merged_df[col] = merged_df[col].round().astype(int)


In [28]:
Train_data = pd.merge(merged_df, final_data_using_stack, on="Time", how="outer")
Train_data

,Time,Incheon_Temp,Incheon_Precipitation,Incheon_Humidity,Incheon_Insolation,Incheon_Cloud,Busan_Temp,Busan_Precipitation,Busan_Humidity,Busan_Insolation,...,Daegu_Precipitation,Daegu_Humidity,Daegu_Insolation,Daegu_Cloud,Seoul_Temp,Seoul_Precipitation,Seoul_Humidity,Seoul_Insolation,Seoul_Cloud,Generation
0,2018-01-01 00:00:00,-1.4,0.1,49.0,0.01,0.0,1.20,0.0,35.0,NaN,...,0.0,37.0,0.00,0.0,-3.2,0.0,40.0,0.010,0.000000,62446
1,2018-01-01 01:00:00,-1.6,0.1,47.0,0.00,0.0,0.65,0.0,33.0,0.02,...,0.0,43.0,0.00,0.0,-3.3,0.0,41.0,0.000,0.000000,59691
2,2018-01-01 02:00:00,-1.9,0.1,47.0,0.00,0.0,0.10,0.0,34.0,0.02,...,0.0,46.0,0.01,0.0,-3.7,0.0,42.0,0.000,0.000000,57231
3,2018-01-01 03:00:00,-1.8,0.1,44.0,0.00,0.0,0.00,0.0,37.0,0.02,...,0.0,47.0,0.01,0.0,-4.0,0.0,44.0,0.000,0.000000,55500
4,2018-01-01 04:00:00,-2.0,0.1,48.0,0.00,0.0,-0.10,0.0,42.0,0.02,...,0.0,47.0,0.01,0.0,-4.2,0.0,53.0,0.000,0.000000,54821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22627,2020-07-31 19:00:00,25.0,0.0,95.0,0.18,10.0,26.10,0.0,86.0,0.22,...,0.0,73.0,0.25,10.0,27.4,0.3,82.0,0.175,9.000000,70347
22628,2020-07-31 20:00:00,24.9,0.0,96.0,0.02,10.0,26.10,0.0,84.0,0.01,...,0.0,72.0,0.02,10.0,27.0,0.3,83.0,0.090,9.000000,70092
22629,2020-07-31 21:00:00,24.9,0.0,96.0,0.02,10.0,26.10,0.0,84.0,0.01,...,0.0,71.0,0.02,8.0,26.7,0.2,86.0,0.090,8.666667,69176
22630,2020-07-31 22:00:00,24.9,0.0,97.0,0.02,10.0,26.10,0.0,85.0,0.01,...,0.0,75.0,0.02,10.0,26.5,0.2,87.0,0.090,8.333333,66569
